In [14]:
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import OneshotPlanner
from unified_planning.engines import PlanGenerationResultStatus

reader = PDDLReader()
problem = reader.parse_problem("domain.pddl", "problem.pddl")

with OneshotPlanner(name="pyperplan") as planner:
    result = planner.solve(problem)

if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
    print("Plan found:")
    for i, action in enumerate(result.plan.actions):
        print(f"{i+1}. {action}")
else:
    print("No plan.")

Plan found:
1. assign-model(gpt4, req1, tok2000, tok8000, coding, openai, price10, budget20, user1)
